In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import tensorflow as tf

# print(os.listdir("../input"))

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

2024-05-14 18:28:37.117253: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 18:28:37.117453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 18:28:37.267146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv("/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [4]:
df['source'].value_counts()

source
tides        50000
ted          39881
indic2012    37726
Name: count, dtype: int64

In [5]:
df

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
...,...,...,...
127602,indic2012,Examples of art deco construction can be found...,आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल ...
127603,ted,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।
127604,tides,"As for the other derivatives of sulphur , the ...","जहां तक गंधक के अन्य उत्पादों का प्रश्न है , द..."
127605,tides,its complicated functioning is defined thus in...,Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .


In [6]:
df.drop('source', axis=1, inplace=True)

In [7]:
df.isnull().sum()

english_sentence    2
hindi_sentence      0
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
def data_cleaning(text):

    # Convert to lower
    text = text.lower()

    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove digits
    text = re.sub(r"\d", '', text)
    
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text)
    
    return text

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 127605 entries, 0 to 127606
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   english_sentence  127605 non-null  object
 1   hindi_sentence    127605 non-null  object
dtypes: object(2)
memory usage: 2.9+ MB


In [11]:
df['cleaned_english_sentence'] = df['english_sentence'].apply(data_cleaning)
df['cleaned_hindi_sentence'] = df['hindi_sentence'].apply(data_cleaning)

In [12]:
# Add start and end tokens to target sequences
df['cleaned_hindi_sentence'] = df['cleaned_hindi_sentence'].apply(lambda x : 'start '+ x + ' end')

In [13]:
df

,english_sentence,hindi_sentence,cleaned_english_sentence,cleaned_hindi_sentence
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...",politicians do not have permission to do what ...,start राजनीतिज्ञों के पास जो कार्य करना चाहिए ...
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...,id like to tell you about one such child,start मई आपको ऐसे ही एक बच्चे के बारे में बतान...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।,this percentage is even greater than the perce...,start यह प्रतिशत भारत में हिन्दुओं प्रतिशत से ...
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते,what we really mean is that theyre bad at not ...,start हम ये नहीं कहना चाहते कि वो ध्यान नहीं द...
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।,the ending portion of these vedas is called up...,start इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता ...
...,...,...,...,...
127602,Examples of art deco construction can be found...,आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल ...,examples of art deco construction can be found...,start आर्ट डेको शैली के निर्माण मैरीन ड्राइव औ...
127603,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।,and put it in our cheeks,start और अपने गालों में डाल लेते हैं। end
127604,"As for the other derivatives of sulphur , the ...","जहां तक गंधक के अन्य उत्पादों का प्रश्न है , द...",as for the other derivatives of sulphur the co...,start जहां तक गंधक के अन्य उत्पादों का प्रश्न ...
127605,its complicated functioning is defined thus in...,Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .,its complicated functioning is defined thus in...,start zरचनाप्रकिया को उसने एक पहेली में यों बा...


In [14]:
# Shuffle the dataframe
# df = df.sample(n=100, random_state=42).reset_index(drop=True)

In [15]:
df.head()

,english_sentence,hindi_sentence,cleaned_english_sentence,cleaned_hindi_sentence
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...",politicians do not have permission to do what ...,start राजनीतिज्ञों के पास जो कार्य करना चाहिए ...
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...,id like to tell you about one such child,start मई आपको ऐसे ही एक बच्चे के बारे में बतान...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।,this percentage is even greater than the perce...,start यह प्रतिशत भारत में हिन्दुओं प्रतिशत से ...
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते,what we really mean is that theyre bad at not ...,start हम ये नहीं कहना चाहते कि वो ध्यान नहीं द...
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।,the ending portion of these vedas is called up...,start इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता ...


In [16]:
df['hindi_sentence'][0], df['cleaned_hindi_sentence'][0]

('राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .',
 'start राजनीतिज्ञों के पास जो कार्य करना चाहिए वह करने कि अनुमति नहीं है  end')

In [17]:
#Tokenize the texts and convert to sequences
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='')
en_tokenizer.fit_on_texts(df['cleaned_english_sentence'])
en_sequences = en_tokenizer.texts_to_sequences(df['cleaned_english_sentence'])

hi_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='')
hi_tokenizer.fit_on_texts(df['cleaned_hindi_sentence'])
hi_sequences = hi_tokenizer.texts_to_sequences(df['cleaned_hindi_sentence'])

english_vocab_size = len(en_tokenizer.word_index) + 1
hindi_vocab_size = len(hi_tokenizer.word_index) + 1
print("English Vocab Size: ", english_vocab_size)
print("Hindi Vocab Size: ", hindi_vocab_size)

English Vocab Size:  72688
Hindi Vocab Size:  81979


In [18]:
en_max_length = df['cleaned_english_sentence'].apply(lambda x:len(str(x).split())).max()
hi_max_length = df['cleaned_hindi_sentence'].apply(lambda x:len(str(x).split())).max()

en_max_length, hi_max_length

(398, 419)

In [19]:
from tensorflow.keras.utils import pad_sequences
encoder_input = pad_sequences(en_sequences, maxlen=en_max_length, padding='post')
decoder_input = pad_sequences(hi_sequences, maxlen=hi_max_length, padding='post')

In [22]:
print("Size taken by the array:", encoder_input.nbytes, "bytes")

size_in_bytes = encoder_input.nbytes
size_in_kilobytes = size_in_bytes / 1024
size_in_megabytes = size_in_kilobytes / 1024
size_in_gigabytes = size_in_megabytes / 1024

print("Size taken by the array:", size_in_bytes, "bytes")
print("Size taken by the array:", size_in_kilobytes, "KB")
print("Size taken by the array:", size_in_megabytes, "MB")
print("Size taken by the array:", size_in_gigabytes, "GB")

Size taken by the array: 203147160 bytes
Size taken by the array: 203147160 bytes
Size taken by the array: 198385.8984375 KB
Size taken by the array: 193.7362289428711 MB
Size taken by the array: 0.18919553607702255 GB


In [177]:
encoder_input

array([[2849,   67,   23, ...,    0,    0,    0],
       [1887,   65,    5, ...,    0,    0,    0],
       [  13, 2850,    8, ...,    0,    0,    0],
       ...,
       [  16,   12,    2, ...,    0,    0,    0],
       [  37, 2447, 2728, ...,    0,    0,    0],
       [3597,  132,  872, ...,    0,    0,    0]], dtype=int32)

In [178]:
decoder_input

array([[    2,  9756,     4, ...,     0,     0,     0],
       [    2,  1091,   158, ...,     0,     0,     0],
       [    2,    17,   231, ...,     0,     0,     0],
       ...,
       [    2,   203,    36, ...,     0,     0,     0],
       [    2, 81977,    11, ...,     0,     0,     0],
       [    2,   554,    22, ...,     0,     0,     0]], dtype=int32)

In [179]:
# hi_tokenizer.word_index
len(hi_tokenizer.word_index)

81978

In [126]:
# hi_tokenizer.word_index

In [24]:
# One-hot encoding input sequences
decoder_target_data = np.zeros((len(df['cleaned_hindi_sentence']), hi_max_length, len(hi_tokenizer.word_index) + 1), dtype='float32')
for i, text in enumerate(df['cleaned_hindi_sentence']):
    for t, token in enumerate(text.split()):
        if t > 0:
            index = hi_tokenizer.word_index[token]
            decoder_target_data[i, t-1, index] = 1.0
    

In [25]:
print("Size taken by the array:", decoder_target_data.nbytes, "bytes")

size_in_bytes = decoder_target_data.nbytes
size_in_kilobytes = size_in_bytes / 1024
size_in_megabytes = size_in_kilobytes / 1024
size_in_gigabytes = size_in_megabytes / 1024

print("Size taken by the array:", size_in_bytes, "bytes")
print("Size taken by the array:", size_in_kilobytes, "KB")
print("Size taken by the array:", size_in_megabytes, "MB")
print("Size taken by the array:", size_in_gigabytes, "GB")

Size taken by the array: 17532519174420 bytes
Size taken by the array: 17532519174420 bytes
Size taken by the array: 17121600756.269531 KB
Size taken by the array: 16720313.238544464 MB
Size taken by the array: 16328.430897016078 GB


In [181]:
latent_dim = 50
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(english_vocab_size, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [182]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(hindi_vocab_size, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(hindi_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [183]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 50)  │  3,634,400 │ input_layer_7[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_4         │ (None, None)      │          0 │ input_layer_7[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, None, 50)  │  4,098,950 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 50),      │     20,200 │ embedding_3[0][0… │
│                     │ (None, 50),       │            │ not_equal_4[0][0] │
│                     │ (None, 50)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, None,     │     20,200 │ embedding_4[0][0… │
│                     │ 50), (None, 50),  │            │ lstm_3[0][1],     │
│                     │ (None, 50)]       │            │ lstm_3[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None,      │  4,180,929 │ lstm_4[0][0]      │
│                     │ 81979)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,954,679 (45.60 MB)

 Trainable params: 11,954,679 (45.60 MB)

 Non-trainable params: 0 (0.00 B)

In [185]:
# [encoder_input[0], decoder_input[0]]

In [187]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
model.fit(
    [encoder_input, decoder_input],
    decoder_target_data,
    epochs=100,
    validation_split=0.2
)

In [135]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [140]:
# from tensorflow.keras.utils import plot_model
# plot_model(decoder_model)

In [146]:
encoder_input[0]

array([ 11, 142,   2, 143,   5, 144,   9,   6,   2, 145, 146,   5,   6,
         2, 147,   3,   2, 148,  47,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [149]:
states_value = encoder_model.predict(encoder_input[[0]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [150]:
states_value

[array([[ 0.7030472 , -0.83324796,  0.7301649 , -0.6235321 , -0.44884267,
          0.6468442 , -0.7333052 , -0.69582283, -0.6322272 , -0.63459927,
          0.7466074 , -0.734724  ,  0.6688732 ,  0.8760372 , -0.6846031 ,
         -0.7382094 , -0.5538812 , -0.67503226,  0.683362  ,  0.5652334 ,
         -0.7910328 , -0.7253302 ,  0.8564563 , -0.7235232 ,  0.7729779 ,
          0.86972696,  0.84596056, -0.82412606,  0.7792142 , -0.8465083 ,
         -0.60422593,  0.7944207 ,  0.5977002 , -0.4621362 , -0.7681987 ,
         -0.72538245,  0.43519884,  0.3745325 ,  0.79378766, -0.81408787,
         -0.5035846 ,  0.7431147 , -0.24964501,  0.47993746, -0.6923111 ,
          0.70535725,  0.44244924,  0.6666138 ,  0.62359804,  0.8541055 ]],
       dtype=float32),
 array([[  3.4713767,  -3.4060228,   3.5745683,  -2.0274491,  -3.830536 ,
           2.3795693,  -2.5660884,  -3.6171024,  -6.10418  ,  -5.064923 ,
           4.3747606,  -3.690565 ,   4.2579336,   3.0999672,  -1.5596956,
          -4.

In [153]:
decoder_input[0][0]

2

In [155]:
output_tokens, h, c = decoder_model.predict([2] + states_value)

ValueError: Unrecognized data type: x=[2, array([[ 0.7030472 , -0.83324796,  0.7301649 , -0.6235321 , -0.44884267,
         0.6468442 , -0.7333052 , -0.69582283, -0.6322272 , -0.63459927,
         0.7466074 , -0.734724  ,  0.6688732 ,  0.8760372 , -0.6846031 ,
        -0.7382094 , -0.5538812 , -0.67503226,  0.683362  ,  0.5652334 ,
        -0.7910328 , -0.7253302 ,  0.8564563 , -0.7235232 ,  0.7729779 ,
         0.86972696,  0.84596056, -0.82412606,  0.7792142 , -0.8465083 ,
        -0.60422593,  0.7944207 ,  0.5977002 , -0.4621362 , -0.7681987 ,
        -0.72538245,  0.43519884,  0.3745325 ,  0.79378766, -0.81408787,
        -0.5035846 ,  0.7431147 , -0.24964501,  0.47993746, -0.6923111 ,
         0.70535725,  0.44244924,  0.6666138 ,  0.62359804,  0.8541055 ]],
      dtype=float32), array([[  3.4713767,  -3.4060228,   3.5745683,  -2.0274491,  -3.830536 ,
          2.3795693,  -2.5660884,  -3.6171024,  -6.10418  ,  -5.064923 ,
          4.3747606,  -3.690565 ,   4.2579336,   3.0999672

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [145]:
encoder_input

array([[ 11, 142,   2, ...,   0,   0,   0],
       [ 10,   7, 149, ...,   0,   0,   0],
       [  7, 151, 152, ...,   0,   0,   0],
       ...,
       [ 14, 775,   8, ...,   0,   0,   0],
       [ 23,  16, 783, ...,   0,   0,   0],
       [793, 794,   7, ...,   0,   0,   0]], dtype=int32)